In [2]:
import pandas as pd
from glob import glob
from sklearn.metrics import roc_auc_score
from analysis.main_analyzer import MainAnalyzer
from analysis.utils import drop_weird_rows
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from analysis.work_counter_analytics import  WordCounterAnalytics

In [3]:
questions = pd.read_csv("vkcup/data.csv", sep=";", index_col="ID")
test = pd.read_csv("vkcup/test.csv", sep=";", index_col="ID")
test['Answer'] = 0
train = pd.read_csv("vkcup/train.csv", sep=";", index_col="ID")
data = questions.join(train)
drop_weird_rows(data)
# data=data[data.index<30000]

In [5]:
main_analyzer = MainAnalyzer()
main_analyzer(data)


/home/ivan/projects/vkcup/analysis/work_counter_analytics.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(x2) / np.mean(x1)


In [6]:
train=data[data.index<=30000]
val=data[(data.index>30000)] 

# add_train = val[val.sure_is_bad==1]
# add_train.Answer = 0
# train = pd.concat([train,add_train])


/home/ivan/projects/vkcup/venv/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
Y = train.Answer
del train['Answer']
X = train

Y_val = val.Answer
del val['Answer']
X_val = val

In [10]:
cat_features=['Question','most_popular_word', 'first_word', 'second_word', 'last_word', 'second_most_popular_word', 'word_after_quote_let']

model = CatBoostClassifier(iterations=4000, 
#                            scale_pos_weight=1/Y.mean(),
                           learning_rate=0.02,
                           task_type="GPU",
                           eval_metric='AUC',
                           cat_features=cat_features,
                          )


model.fit(X,
          Y,
          eval_set=(X_val, Y_val),
          verbose=True)


0:	learn: 0.7814793	test: 0.7477381	best: 0.7477381 (0)	total: 52.8ms	remaining: 3m 31s
1:	learn: 0.8046862	test: 0.7698683	best: 0.7698683 (1)	total: 102ms	remaining: 3m 24s
2:	learn: 0.8171117	test: 0.7878527	best: 0.7878527 (2)	total: 148ms	remaining: 3m 17s
3:	learn: 0.8215558	test: 0.7974991	best: 0.7974991 (3)	total: 192ms	remaining: 3m 12s
4:	learn: 0.8270866	test: 0.8042049	best: 0.8042049 (4)	total: 235ms	remaining: 3m 7s
5:	learn: 0.8272871	test: 0.8051476	best: 0.8051476 (5)	total: 298ms	remaining: 3m 18s
6:	learn: 0.8288562	test: 0.8070478	best: 0.8070478 (6)	total: 337ms	remaining: 3m 12s
7:	learn: 0.8284227	test: 0.8054953	best: 0.8070478 (6)	total: 373ms	remaining: 3m 6s
8:	learn: 0.8316029	test: 0.8061095	best: 0.8070478 (6)	total: 414ms	remaining: 3m 3s
9:	learn: 0.8336529	test: 0.8041166	best: 0.8070478 (6)	total: 453ms	remaining: 3m
10:	learn: 0.8354730	test: 0.8052817	best: 0.8070478 (6)	total: 495ms	remaining: 2m 59s
11:	learn: 0.8383865	test: 0.8091550	best: 0.809

94:	learn: 0.8612247	test: 0.8290531	best: 0.8290705 (92)	total: 3.89s	remaining: 2m 39s
95:	learn: 0.8613102	test: 0.8292875	best: 0.8292875 (95)	total: 3.92s	remaining: 2m 39s
96:	learn: 0.8614484	test: 0.8294314	best: 0.8294314 (96)	total: 3.96s	remaining: 2m 39s
97:	learn: 0.8616264	test: 0.8294761	best: 0.8294761 (97)	total: 4s	remaining: 2m 39s
98:	learn: 0.8616428	test: 0.8295144	best: 0.8295144 (98)	total: 4.04s	remaining: 2m 39s
99:	learn: 0.8616446	test: 0.8296409	best: 0.8296409 (99)	total: 4.08s	remaining: 2m 39s
100:	learn: 0.8618146	test: 0.8298599	best: 0.8298599 (100)	total: 4.13s	remaining: 2m 39s
101:	learn: 0.8618207	test: 0.8298352	best: 0.8298599 (100)	total: 4.17s	remaining: 2m 39s
102:	learn: 0.8620474	test: 0.8302257	best: 0.8302257 (102)	total: 4.22s	remaining: 2m 39s
103:	learn: 0.8621540	test: 0.8301848	best: 0.8302257 (102)	total: 4.26s	remaining: 2m 39s
104:	learn: 0.8621381	test: 0.8301870	best: 0.8302257 (102)	total: 4.3s	remaining: 2m 39s
105:	learn: 0.8

186:	learn: 0.8697727	test: 0.8349955	best: 0.8349955 (186)	total: 7.49s	remaining: 2m 32s
187:	learn: 0.8697930	test: 0.8349627	best: 0.8349955 (186)	total: 7.53s	remaining: 2m 32s
188:	learn: 0.8698387	test: 0.8350046	best: 0.8350046 (188)	total: 7.57s	remaining: 2m 32s
189:	learn: 0.8699266	test: 0.8350656	best: 0.8350656 (189)	total: 7.61s	remaining: 2m 32s
190:	learn: 0.8699216	test: 0.8350165	best: 0.8350656 (189)	total: 7.64s	remaining: 2m 32s
191:	learn: 0.8700133	test: 0.8351591	best: 0.8351591 (191)	total: 7.68s	remaining: 2m 32s
192:	learn: 0.8700246	test: 0.8351601	best: 0.8351601 (192)	total: 7.71s	remaining: 2m 32s
193:	learn: 0.8700324	test: 0.8351876	best: 0.8351876 (193)	total: 7.75s	remaining: 2m 32s
194:	learn: 0.8700640	test: 0.8352748	best: 0.8352748 (194)	total: 7.79s	remaining: 2m 32s
195:	learn: 0.8700894	test: 0.8352492	best: 0.8352748 (194)	total: 7.83s	remaining: 2m 32s
196:	learn: 0.8701331	test: 0.8353283	best: 0.8353283 (196)	total: 7.91s	remaining: 2m 32s

281:	learn: 0.8755256	test: 0.8389614	best: 0.8389614 (281)	total: 11.3s	remaining: 2m 28s
282:	learn: 0.8756263	test: 0.8389923	best: 0.8389923 (282)	total: 11.3s	remaining: 2m 28s
283:	learn: 0.8756426	test: 0.8390200	best: 0.8390200 (283)	total: 11.4s	remaining: 2m 28s
284:	learn: 0.8757476	test: 0.8390653	best: 0.8390653 (284)	total: 11.4s	remaining: 2m 28s
285:	learn: 0.8758168	test: 0.8391515	best: 0.8391515 (285)	total: 11.4s	remaining: 2m 28s
286:	learn: 0.8758658	test: 0.8391985	best: 0.8391985 (286)	total: 11.5s	remaining: 2m 28s
287:	learn: 0.8759553	test: 0.8393019	best: 0.8393019 (287)	total: 11.5s	remaining: 2m 28s
288:	learn: 0.8760423	test: 0.8393250	best: 0.8393250 (288)	total: 11.6s	remaining: 2m 28s
289:	learn: 0.8762157	test: 0.8394524	best: 0.8394524 (289)	total: 11.6s	remaining: 2m 28s
290:	learn: 0.8762935	test: 0.8395244	best: 0.8395244 (290)	total: 11.6s	remaining: 2m 28s
291:	learn: 0.8763832	test: 0.8395480	best: 0.8395480 (291)	total: 11.7s	remaining: 2m 28s

374:	learn: 0.8800291	test: 0.8422896	best: 0.8422896 (374)	total: 14.8s	remaining: 2m 23s
375:	learn: 0.8800834	test: 0.8423451	best: 0.8423451 (375)	total: 14.8s	remaining: 2m 23s
376:	learn: 0.8800826	test: 0.8423514	best: 0.8423514 (376)	total: 14.9s	remaining: 2m 23s
377:	learn: 0.8800816	test: 0.8423646	best: 0.8423646 (377)	total: 14.9s	remaining: 2m 22s
378:	learn: 0.8802013	test: 0.8424194	best: 0.8424194 (378)	total: 15s	remaining: 2m 22s
379:	learn: 0.8802215	test: 0.8424239	best: 0.8424239 (379)	total: 15s	remaining: 2m 22s
380:	learn: 0.8802992	test: 0.8424785	best: 0.8424785 (380)	total: 15s	remaining: 2m 22s
381:	learn: 0.8803001	test: 0.8424785	best: 0.8424785 (380)	total: 15.1s	remaining: 2m 23s
382:	learn: 0.8802904	test: 0.8424898	best: 0.8424898 (382)	total: 15.2s	remaining: 2m 23s
383:	learn: 0.8803474	test: 0.8425075	best: 0.8425075 (383)	total: 15.2s	remaining: 2m 23s
384:	learn: 0.8803781	test: 0.8425562	best: 0.8425562 (384)	total: 15.2s	remaining: 2m 22s
385:	

...

3564:	learn: 0.9329231	test: 0.8553875	best: 0.8554082 (3554)	total: 2m 21s	remaining: 17.3s
3565:	learn: 0.9329321	test: 0.8553917	best: 0.8554082 (3554)	total: 2m 21s	remaining: 17.3s
3566:	learn: 0.9329507	test: 0.8554021	best: 0.8554082 (3554)	total: 2m 21s	remaining: 17.2s
3567:	learn: 0.9329545	test: 0.8554051	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17.2s
3568:	learn: 0.9329546	test: 0.8554053	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17.2s
3569:	learn: 0.9329537	test: 0.8554060	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17.1s
3570:	learn: 0.9329536	test: 0.8554060	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17.1s
3571:	learn: 0.9329540	test: 0.8554045	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17s
3572:	learn: 0.9329550	test: 0.8554043	best: 0.8554082 (3554)	total: 2m 22s	remaining: 17s
3573:	learn: 0.9329614	test: 0.8554132	best: 0.8554132 (3573)	total: 2m 22s	remaining: 17s
3574:	learn: 0.9329616	test: 0.8554150	best: 0.8554150 (3574)	total: 2m 22s	

3656:	learn: 0.9335746	test: 0.8555351	best: 0.8555351 (3656)	total: 2m 25s	remaining: 13.7s
3657:	learn: 0.9335859	test: 0.8555276	best: 0.8555351 (3656)	total: 2m 25s	remaining: 13.6s
3658:	learn: 0.9335861	test: 0.8555284	best: 0.8555351 (3656)	total: 2m 25s	remaining: 13.6s
3659:	learn: 0.9335910	test: 0.8555371	best: 0.8555371 (3659)	total: 2m 25s	remaining: 13.6s
3660:	learn: 0.9336128	test: 0.8555281	best: 0.8555371 (3659)	total: 2m 25s	remaining: 13.5s
3661:	learn: 0.9336196	test: 0.8555263	best: 0.8555371 (3659)	total: 2m 25s	remaining: 13.5s
3662:	learn: 0.9336223	test: 0.8555183	best: 0.8555371 (3659)	total: 2m 26s	remaining: 13.4s
3663:	learn: 0.9336386	test: 0.8555351	best: 0.8555371 (3659)	total: 2m 26s	remaining: 13.4s
3664:	learn: 0.9336368	test: 0.8555350	best: 0.8555371 (3659)	total: 2m 26s	remaining: 13.4s
3665:	learn: 0.9336568	test: 0.8555283	best: 0.8555371 (3659)	total: 2m 26s	remaining: 13.3s
3666:	learn: 0.9336599	test: 0.8555320	best: 0.8555371 (3659)	total: 2

3746:	learn: 0.9343500	test: 0.8555486	best: 0.8555861 (3727)	total: 2m 29s	remaining: 10.1s
3747:	learn: 0.9343635	test: 0.8555254	best: 0.8555861 (3727)	total: 2m 29s	remaining: 10s
3748:	learn: 0.9343874	test: 0.8555238	best: 0.8555861 (3727)	total: 2m 29s	remaining: 10s
3749:	learn: 0.9343881	test: 0.8555263	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.97s
3750:	learn: 0.9343885	test: 0.8555255	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.93s
3751:	learn: 0.9344043	test: 0.8555068	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.89s
3752:	learn: 0.9344074	test: 0.8555106	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.85s
3753:	learn: 0.9344085	test: 0.8555090	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.81s
3754:	learn: 0.9344136	test: 0.8555062	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.77s
3755:	learn: 0.9344164	test: 0.8554999	best: 0.8555861 (3727)	total: 2m 29s	remaining: 9.73s
3756:	learn: 0.9344422	test: 0.8555178	best: 0.8555861 (3727)	total: 2m 29

3838:	learn: 0.9350694	test: 0.8554645	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.43s
3839:	learn: 0.9350894	test: 0.8554600	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.39s
3840:	learn: 0.9351092	test: 0.8554500	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.35s
3841:	learn: 0.9351105	test: 0.8554503	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.31s
3842:	learn: 0.9351125	test: 0.8554500	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.27s
3843:	learn: 0.9351295	test: 0.8554328	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.23s
3844:	learn: 0.9351318	test: 0.8554293	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.19s
3845:	learn: 0.9351320	test: 0.8554299	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.15s
3846:	learn: 0.9351318	test: 0.8554299	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.11s
3847:	learn: 0.9351446	test: 0.8554434	best: 0.8555861 (3727)	total: 2m 33s	remaining: 6.07s
3848:	learn: 0.9351661	test: 0.8554197	best: 0.8555861 (3727)	total: 2

3929:	learn: 0.9358174	test: 0.8554345	best: 0.8555861 (3727)	total: 2m 36s	remaining: 2.79s
3930:	learn: 0.9358279	test: 0.8554398	best: 0.8555861 (3727)	total: 2m 36s	remaining: 2.75s
3931:	learn: 0.9358296	test: 0.8554386	best: 0.8555861 (3727)	total: 2m 36s	remaining: 2.71s
3932:	learn: 0.9358334	test: 0.8554406	best: 0.8555861 (3727)	total: 2m 36s	remaining: 2.67s
3933:	learn: 0.9358382	test: 0.8554544	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.63s
3934:	learn: 0.9358472	test: 0.8554479	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.59s
3935:	learn: 0.9358591	test: 0.8554482	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.55s
3936:	learn: 0.9358647	test: 0.8554734	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.51s
3937:	learn: 0.9358678	test: 0.8554733	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.47s
3938:	learn: 0.9358732	test: 0.8554866	best: 0.8555861 (3727)	total: 2m 37s	remaining: 2.43s
3939:	learn: 0.9358926	test: 0.8554794	best: 0.8555861 (3727)	total: 2

In [9]:
val['predict'] = model.predict_proba(val)[:,1]

/home/ivan/projects/vkcup/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
val[['predict']].to_csv('predicts5.csv',header=False)